#                                      Random Forest

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pickle
import requests, json
%matplotlib inline

In [5]:
df = pd.read_csv('loan_data.csv')

In [6]:
df.head(1)
# new_df = final_df.astype(int)
# new_df
# final_df.info()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.1,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier


In [8]:
df2 = df.drop(['purpose'],axis=1)
df2.head(1)

,credit.policy,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,0.1189,829.1,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0


In [9]:
X = df2.drop('not.fully.paid',axis=1)
y = df2['not.fully.paid']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [10]:
# Recursive Feature Elimination
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# load the iris datasets
dataset = df2
# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 6)
rfe = rfe.fit(X_train, y_train)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[ True  True False  True False False False False False  True  True  True]
[1 1 5 1 3 4 6 7 2 1 1 1]


In [ ]:
# categorical_feats = ['purpose']
# final_df = pd.get_dummies(df,columns=categorical_feats,drop_first=True)

In [55]:
# Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
# import pandas
# import numpy
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
# # load data
# # url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"
# # names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
# # dataframe = pandas.read_csv(url, names=names)
# # array = dataframe.values
# # X = array[:,0:8]
# # Y = array[:,8]
# # feature extraction
# test = SelectKBest(score_func=chi2, k=4)
# fit = test.fit(X_train, y_train)
# # summarize scores
# numpy.set_printoptions(precision=3)
# print(fit.scores_)
# features = fit.transform(X_train)
# # summarize selected features
# print(features[0:5,:])

In [56]:
# # Feature Importance
# from sklearn import datasets
# from sklearn import metrics
# from sklearn.ensemble import ExtraTreesClassifier
# # load the iris datasets
# dataset = final_df
# # fit an Extra Trees model to the data
# model = ExtraTreesClassifier()
# model.fit(X, y)
# # display the relative importance of each attribute
# print(model.feature_importances_)

In [11]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [12]:
# save the random forest classification model
rffilename = 'rf_loan_classification2.pkl'
pickle.dump(rfc, open(rffilename, 'wb'))
# load the model
loaded_model = pickle.load(open(rffilename, 'rb'))

In [13]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,loaded_model.predict(X_test)))

             precision    recall  f1-score   support

          0       0.85      0.99      0.92      2431
          1       0.50      0.03      0.06       443

avg / total       0.80      0.85      0.78      2874



In [15]:
url = "http://localhost:7000/hello"
data = json.dumps({'credit_policy':1,
                   'int_rate':0.1496,  
                   'installment':194.02,
                   'log_annual_inc':10.714418,
                   'dti':4.0,
                   'fico':667,
                   'days_with_cr_line':3180.04, 
                   'revol_bal':3839, 
                   'revol_util' :76.8, 
                   'inq_last_6mths': 0,
                   'delinq_2yrs' :0, 
                   'pub_rec': 1, 
                   'purpose_credit_card': 0, 
                   'purpose_debt_consolidation': 1,
                   'purpose_educational': 0, 
                   'purpose_home_improvement':0,
                   'purpose_major_purchase':0,  
                   'purpose_small_business' :0})

r =  requests.post(url, data=data)
raw = r.json()
print (raw)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Data Preprocessing

In [22]:
# Feature Extraction with RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# feature extraction
model = LogisticRegression()
rfe = RFE(model, 3)
fit = rfe.fit(X_train, y_train)
print(fit.n_features_)
print(fit.support_) 
print(fit.ranking_)
# true ou 1.

3
[ True  True False False False False False False False False False  True]
[ 1  1  8  2  6  7  9 10  5  3  4  1]


In [19]:
# Rescale data (between 0 and 1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X_train)
# summarize transformed data
numpy.set_printoptions(precision=3)
print(rescaledX[0:5,:])

[[ 1.     0.528  0.577  0.482  0.49   0.333  0.217  0.008  0.195  0.     0.
   0.   ]
 [ 1.     0.111  0.321  0.498  0.029  0.762  0.287  0.002  0.055  0.     0.
   0.   ]
 [ 0.     0.588  0.209  0.448  0.618  0.143  0.177  0.006  0.644  0.121
   0.077  0.   ]
 [ 1.     0.502  0.536  0.517  0.425  0.333  0.186  0.017  0.61   0.061  0.
   0.   ]
 [ 1.     0.293  0.124  0.53   0.594  0.333  0.352  0.015  0.576  0.061  0.
   0.   ]]


In [18]:
# # Standardize data (0 mean, 1 stdev)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train, y_train)
rescaledX = scaler.transform(X_train, y_train)
# summarize transformed data
numpy.set_printoptions(precision=3)
print(rescaledX[0:5,:])

[[ 0.494  0.74   1.1   -0.359  0.305 -0.757 -0.238 -0.226 -0.812 -0.722
  -0.298 -0.233]
 [ 0.494 -1.68  -0.039 -0.186 -1.701  1.599  0.258 -0.432 -1.387 -0.722
  -0.298 -0.233]
 [-2.022  1.089 -0.537 -0.723  0.866 -1.805 -0.516 -0.29   1.028  1.094
   1.465 -0.233]
 [ 0.494  0.588  0.921  0.028  0.025 -0.757 -0.456  0.136  0.89   0.186
  -0.298 -0.233]
 [ 0.494 -0.622 -0.914  0.165  0.761 -0.757  0.718  0.047  0.749  0.186
  -0.298 -0.233]]


In [20]:
# Normalize data (length of 1)
from sklearn.preprocessing import Normalizer
scaler = Normalizer().fit(X_train)
normalizedX = scaler.transform(X_train)
# summarize transformed data
numpy.set_printoptions(precision=3)
print(normalizedX[0:5,:])

[[  1.004e-04   1.432e-05   5.514e-02   1.076e-03   1.474e-03   6.850e-02
    3.978e-01   9.133e-01   2.330e-03   0.000e+00   0.000e+00   0.000e+00]
 [  1.751e-04   1.355e-05   5.467e-02   1.895e-03   1.506e-04   1.352e-01
    9.088e-01   3.908e-01   1.138e-03   0.000e+00   0.000e+00   0.000e+00]
 [  0.000e+00   1.967e-05   2.699e-02   1.357e-03   2.397e-03   8.306e-02
    4.231e-01   9.018e-01   9.911e-03   5.175e-04   1.294e-04   0.000e+00]
 [  4.672e-05   6.470e-06   2.390e-02   5.116e-04   5.952e-04   3.186e-02
    1.598e-01   9.863e-01   3.392e-03   9.343e-05   0.000e+00   0.000e+00]
 [  5.194e-05   5.501e-06   6.762e-03   5.732e-04   9.251e-04   3.543e-02
    3.288e-01   9.437e-01   3.558e-03   1.039e-04   0.000e+00   0.000e+00]]


In [21]:
# binarization
from sklearn.preprocessing import Binarizer

binarizer = Binarizer(threshold=0.0).fit(X_train)
binaryX = binarizer.transform(X_train)
# summarize transformed data
numpy.set_printoptions(precision=3)
print(binaryX[0:5,:])

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.]
 [ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.]]
